In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor 
import pickle

In [5]:
df=pd.read_csv('weather_forecast.csv')

In [6]:
df.drop(columns=['Unnamed: 0', 'plant_id','datetime_utc'],inplace=True)
df[df==-9999]=np.nan
df.drop(columns=['wind_chill', 'heat_index','qpf','snow','pop','fctcode','precip_accumulation'],inplace=True)
df.cloud_cover=df.cloud_cover.interpolate()
df.wind_bearing=df.wind_bearing.interpolate()
df.wind_speed=df.wind_speed.interpolate()
df.wind_gust=df.wind_gust.interpolate()
df.pressure=df.pressure.interpolate()
df.uv_index=df.uv_index.interpolate()
df.ozone=df.ozone.interpolate()
df.precip_intensity=df.precip_intensity.interpolate()
df.precip_probability=df.precip_probability.interpolate()
df.precip_type=df.precip_type.interpolate()
df.visibility=df.visibility.interpolate()
df.drop(columns=['precip_type'],inplace=True)
df.drop(columns=['datetime_local','sunrise','sunset','updated_at'],inplace=True)
df=df.fillna(df.mean())
df.drop(columns=['icon'],inplace=True)

In [8]:
one_hot = pd.get_dummies(df['summary'])
df = df.drop('summary',axis = 1)
df = df.join(one_hot)

In [9]:
RFmodel = pickle.load(open('RFmodel.sav', 'rb'))

In [12]:
df.columns

Index(['cloud_cover', 'apparent_temperature', 'temperature', 'humidity',
       'dew_point', 'wind_bearing', 'wind_speed', 'wind_gust', 'pressure',
       'uv_index', 'ozone', 'precip_intensity', 'precip_probability',
       'visibility', 'Clear', 'Humid and Mostly Cloudy', 'Mostly Cloudy',
       'Overcast', 'Partly Cloudy'],
      dtype='object')

In [14]:
df.shape

(648, 19)

In [15]:
tpdf=pd.DataFrame(np.zeros((648,19)),columns=['Breezy and Foggy', 'Clear', 'Foggy', 'Humid',
       'Humid and Foggy', 'Humid and Mostly Cloudy', 'Humid and Overcast',
       'Humid and Partly Cloudy', 'Light Rain', 'Light Rain and Humid',
       'Mostly Cloudy', 'Overcast', 'Partly Cloudy', 'Possible Drizzle',
       'Possible Drizzle and Humid', 'Possible Light Rain',
       'Possible Light Rain and Humid', 'Rain', 'Rain and Humid'])

In [23]:
df[['Breezy and Foggy', 'Foggy', 'Humid','Humid and Foggy', 'Humid and Overcast','Humid and Partly Cloudy', 'Light Rain', 'Light Rain and Humid','Possible Drizzle','Possible Drizzle and Humid', 'Possible Light Rain','Possible Light Rain and Humid', 'Rain', 'Rain and Humid']]=tpdf[['Breezy and Foggy', 'Foggy', 'Humid','Humid and Foggy', 'Humid and Overcast','Humid and Partly Cloudy', 'Light Rain', 'Light Rain and Humid','Possible Drizzle','Possible Drizzle and Humid', 'Possible Light Rain','Possible Light Rain and Humid', 'Rain', 'Rain and Humid']]

In [26]:
predicted_values=RFmodel.predict(df)

In [27]:
df['power']=predicted_values

In [28]:
df.to_csv('OUTPUT.csv')